# INF200 Lecture No J02
### Hans Ekkehard Plesser / NMBU
### 7 January 2019

## Today's topics
- Class, static, private methods
- Repetition: Mutables as default arguments
- Team repositories and branches

## Class methods

- Methods usually work on individual objects
- Sometimes, it can be useful to do things at a class level
- Examples
    - count number of instances of a class
    - set parameters that apply to all members of a class
- We can achieve this by writing *class methods*
- A method becomes a class method by adding the `@classmethod` decorator
- The `self` argument is replaced by `cls` in class methods

In [5]:
class Truck:
    
    instance_count = 0       # number of trucks
    weight_empty = 1000      # weight of empty truck
    
    @classmethod
    def count_new_truck(cls):
        cls.instance_count += 1
        
    @classmethod
    def num_trucks(cls):
        return cls.instance_count
    
    @classmethod
    def set_weight_empty(cls, we):
        cls.weight_empty = we
        
    def __init__(self, load):
        self._load = load
        self.count_new_truck()
        
    def total_weight(self):
        return self._load + self.weight_empty
    
Truck.set_weight_empty(1500)
trucks = [Truck(load) for load in [100, 500, 1000]]
print("Number of trucks:", Truck.num_trucks())

for truck in trucks:
    print("Total weight:", truck.total_weight())

print("New empty weight")
Truck.set_weight_empty(2000)
for truck in trucks:
    print("Total weight:", truck.total_weight())

trucks[0].weight_empty = 800

print("New empty weight")
Truck.set_weight_empty(2000)
for truck in trucks:
    print("Total weight:", truck.total_weight())


Number of trucks: 3
Total weight: 1600
Total weight: 2000
Total weight: 2500
New empty weight
Total weight: 2100
Total weight: 2500
Total weight: 3000
New empty weight
Total weight: 900
Total weight: 2500
Total weight: 3000


Note the following:

- We can access class attributes through `self`
- When counting new trucks, we must make sure that we update the class attribute `instance_count`,  not create an `instance_count` attribute in the instance created. Therefore, we use the *class* method `count_new_truck()`.
- When calling `self.count_new_truck()`, Python automatically makes sure that the class of `self`,  not `self` is passed as parameter `cls`.

### Class methods and inheritance

- The `cls` argument passed to a class method is always the concrete class of the object on which the class method is called

In [8]:
class A:
    _info = None
    
    @classmethod
    def print_info(cls):
        print("Class info:", cls._info)
        
    def display(self):
        print("Displaying ...", end=' ')
        self.print_info()
        
class B(A):
    _info = "This is class B"
    
class C(A):
    _info = "This is class C"
    
class D(C):
    _info = "This is class D"
    
b, c, d = B(), C(), D()
b.info = ""

##### Call `print_info()` on instances

In [9]:
b.print_info()
c.print_info()
d.print_info()

Class info: This is class B
Class info: This is class C
Class info: This is class D


##### Call `display()` on instances, which then calls `print_info()`

In [4]:
b.display()
c.display()
d.display()

Displaying ... Class info: This is class B
Displaying ... Class info: This is class C
Displaying ... Class info: This is class D


- `cls._info` always resolves to the `_info` class attribute defined in the concrete class to which the instance belongs.

### Static methods

- Sometimes, it can be useful to have a function in a class that behaves as a normal function, i.e., does not need any access to "self". 
- In some cases, one will define such a function outside the class.
- In other cases, it can be useful to define the function inside the class to show where it belongs logically.
- Static methods are used for this purpose. They are defined using the `@staticmethod` decorator.
- Note that they only get passed the arguments explicitly given in the call, no `self` is inserted anywhere.

In [10]:
import random

class Game:
    
    def __init__(self, seed):
        random.seed(seed)
        self.results = []
        
    def play(self):
        n1 = random.random()
        n2 = random.random()
        n3 = random.random()
        res = self._median(n1, n2, n3)
        self.results.append(res)
        
    @staticmethod    
    def _median(a, b, c):
        return sorted([a, b, c])[1]
    
g = Game(12345)
for _ in range(10):
    g.play()
    
g.results

[0.41661987254534116,
 0.2986398551995928,
 0.1616878239293682,
 0.4329362680099159,
 0.5532210855693298,
 0.412119392939301,
 0.5039353681100375,
 0.18997137872182035,
 0.9674824588798714,
 0.7445300401410346]

### Private methods

- Sometimes, it is useful to define "helper" methods that should be used only by other methods of the same class
- To mark these methods as private, start the method name with `_`, e.g. `_helper(self, ...)`


## Mutables as default arguments

- Mutables should **never** be used as default arguments
- Reason: The default value is a mutable object created when the function or method is defined. Therefore, all calls of the method will receive **the same mutable object** as argument
- [Example on Python Tutor](http://pythontutor.com/visualize.html#code=class%20A%3A%0A%20%20%20%20def%20__init__%28self,%20data%3D%5B%5D%29%3A%0A%20%20%20%20%20%20%20%20self.data%20%3D%20data%0A%20%20%20%20def%20add%28self,%20new_data%29%3A%0A%20%20%20%20%20%20%20%20self.data.extend%28new_data%29%0A%0Aa%20%3D%20A%28%5B1,%202,%203%5D%29%0Aa.add%28%5B'a',%20'b'%5D%29%0A%20%20%20%20%20%20%20%20%0Ab%20%3D%20A%28%29%0Ab.add%28%5B'c',%20'd'%5D%29%0A%0Ac%20%3D%20A%28%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false) (see what happens when `c` is created!)

- Solution: Use `None` as default value and create new mutable in function if necessary

In [11]:
class A:
    def __init__(self, data=None):
        if data is None:
            data = []
        self.data = data

    def add(self, new_data):
        self.data.extend(new_data)
        
a = A([1, 2, 3])
a.add(['a', 'b'])

b = A()
b.add(['c', 'd'])

c = A()

a.data, b.data, c.data

([1, 2, 3, 'a', 'b'], ['c', 'd'], [])

## Team repositories and branches

### The basic rule 
**Never force a push**

#### Solve errors *(a)*

1. Pull changes from repository
1. Merge if necessary (GitKraken/Git will do this automatically for you)
    1. Resolve merge conflicts and commit resolved state
1. Push

##### Try this now

###### Round 1
- Both of you make changes to `README.md`, one at the top and one at the bottom of the file
- Try pushing
- If you get an error, see *(a)*
- Continue until both your changes are pushed, and both of you have the changes on your computers

###### Round 2
- Begin as in round 1, but make incompatible changes to the same line of `README.md`
- Try pushing, see *(a)* in case of error
- Continue until both your changes are pushed, and both of you have the changes on your computers

### Local branches

- You can create branches on your computer as you like
- You *should* create branches to test out new things
- Once you are happy with your changes, merge them back into your `master`, then push

##### Try this now
- Create a local branch and make changes in `README.md` in the branch (both of you), commit
- Checkout `master`, each make different changes in master at different ends of file, commit, push
- Both of you should now have each others changes in `master`, plus you own changes in your local branch
- Merge your local branch into `master` and close the local branch
- Push and pull, merge if neccessary, so both in the end have changes from both local branches

#### Long-lived local branches

- Sometimes, local branches can live for quite a while
- Meanwhile, `master` will evolve
- It is then often useful to update the branch with changes in `master`
- To keep the branch up to date with changes in `master`
    1. make sure the branch is checked out
    1. in GitKraken, right-click on `master` and choose `Merge master into <your branch>`
    1. resolve merge conflicts if necessary
    
##### Try this now
In this exercise, one of you, B, works with a local branch, the other, M, with `master`. 

1. B creates a local branch, makes changes in `README.md` in the branch
1. M makes changes in `README.md` in `master` and pushes
1. B pulls changes from `master` into her branch (may need to fetch changes in `master` from GitHub)
1. B makes changes more changes in her branch
1. Repeat 2.-4. a few times
1. B merges final state of her branch into master and pushes.

Now repeat this exercise with opposite roles.


### Remote branches

- You can share branches within you team by pushing them to GitHub
- To work with a remote branch pushed by another team member, you need to check it out locally
- By default, GitHub will automatically set up the local branch to *track* the remote branch
    - You can easily pull changes from and push changes to the remote branch
- Essentially, you work with such a branch as with `master`

##### Try this now
1. Each of you creates a local branch (choose different names!)
1. Create files `README_<branchname>.md` in your respective branches, fill them with some text
1. Push your branches to GitHub
1. Check out each other's branches locally
1. Make changes both in the `README_<branchname>.md` file in your own branch and your partner's branch
1. Push those changes, and pull those that you partner made
1. Repeat 5.-6. a few times
1. Merge all branches into `master`


##### Tidy up your team repository now
- Delete all branches you created during these exercises
- Checkout `master`
- Delete the `README_<branchname>.md` files
- Edit your `README.md` file so that it gives a brief introduction to your team and project.